# Managing agents in Agentspace (using the API)  


In [ ]:
import os

### Authentication
Mostly for Google Colab.  
You will be authenticated with your user in Colab Enterprise (Vertex AI)

In [ ]:
import sys

if "google.colab" in sys.modules:
    print("Running in Google Colab environment")
    from google.colab import auth
    auth.authenticate_user()

This notebook is a running version of the following API https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents   

You can also use the client library, for example in Python: https://cloud.google.com/python/docs/reference/discoveryengine/latest

## Pre-requisites:    
### Agentspace
- Have a project with Agentspace available
- Have an existing Agentspace application - you will need the application ID  
- You can create and manage Agentspace apps in the ***AI Application > Apps*** in the Cloud Console.  
  
### Agent & Agent Engine
- Have an agent deployed to Agent engine  *in the same project* than Agentspace    
- You can create and manage Agent Engine agents in the ***Vertex AI > Agent Engine*** in the Cloud Console.   
- To find information like the name for an agent, click on the agent in the list and choose for example "Deployment Details" on the top of the screen

**Note**: The REST resource for a running agent in agent engine is using the now deprecated *reasonning engine*  name  


## Agent in Agentspace

Publish (and activated agents) in agentspace appears under the Agents menu in Agentspace.  
There are 3 ways to deploy agents. Generally speaking, you must manage the agents in agentspace through the channel you used to publish them.

1. **No code agents created in the agent space web app**
  1. Theses can be administrated directly in the agent space webapp via the 3 dots menu
  1. They will be listed as "PRIVATE"

1. **Agents registred in the Cloud Console UI**
Per August there is a private preview "agents" entry in the left menu - this is where you will manage your agents in the future.

In additon, there is an existing UI to publish Dialog Flow agents. Even though it works for agentEngine agents per medio Aug 2025, it might break or have unexpected side effects - expect changes in the futur
  1. Choose you Agentspace app under the AI Application > app  
  1. Choose configurations, you are now at: *Apps > my-app > Configurations*
  1. Select now the "Assistant" tab.
  1. The agents published through the cloud console are listed under the Agents section "From your org"
  1. You can edit or delete deployed agents in the console. This include changing the "backing" agent engine agent
  
1. **Publish and manage agents with the API**
  1. this is detailed below
  1. They will also come under the "From your organization" section






# Using the Agentspace service API




### Core elements
* Google Project settings like project id, location and the name of the agentspace app
The project id to use for the agents. Note that a lot of resources used earlier project number but support now project ID (which is nicer to work with for humans)  
The location is a Google Cloud Region
* AI setting like which model to use
* The "agent" is a central element when using the discoveryengine.googleapis.com API  
You can set up values for the Agent element in the form below - these values will be used in the whole notebook so come here and update as needed.  

*Agent fields*  
Note that fields at the top level are Agentspace fields.  
The actual agent fields are defined in the Definition field.

The agent resource documentation at https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents#Agent  

- `name`: the fully qualified name of the agent in AgentSpace   
ex: projects/{project}/locations/{location}/collections/{collection}/engines/{engine}/assistants/{assistant}/agents/{agent}

* `DisplayName`: The user friendly name is
* `Definition`: One of  AdkAgentDefinition, ManagedAgentDefinition and A2AAgentDefinition. We use ADK agents in this notebook. --> This is where you find the URI of the agent in AgentEngine


In [ ]:
#@title An agent definition for Agentspace
#@markdown _How your agent will be called in Agentspace_
DISPLAY_NAME = "the Tuesday agent"  #@param {type:"string"}
#@markdown _The description for your agent in Agentspace_
DESCRIPTION = "This agent is created with the API"  #@param {type:"string"}

os.environ["DISPLAY_NAME"] = DISPLAY_NAME
os.environ["DESCRIPTION"] = DESCRIPTION


### Google Cloud Settings  
These are the Google Cloud settings like project, agentspace app, etc

In [ ]:
# @title Settings
#@markdown ---
#@markdown ### Configure your settings below:

#@markdown _What is the ID of your Google Cloud project._

PROJECT_ID = "my-project" #@param {type:"string"}
#@markdown _The Google Cloud region you are using_
LOCATION = "us-central1"  #@param {type:"string"}
#@markdown _The ID of your existing Agentspace application_
AS_APP="XXX" #@param {type:"string"}
#@markdown _The model to use is_
MODEL = "gemini-2.0-flash-001" #@param ["gemini-2.0-pro-exp-02-05","gemini-1.5-pro-002","gemini-1.5-flash-001","gemini-2.0-flash-001"]


import os
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["AS_APP"] = AS_APP
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION


## Create an agent in Agentspace


* name is the fully qualified resource name of the agent *in Agentspace* --> This will be outputed after creation.
* not to be confused with the actual running agent wich must be defined in an agent definition (here with an ADK agent).  





### API documentation  
You Create a resource by sending a `POST` request to the agent resource.  
Doc: https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents/create  

In [ ]:
#@markdown The URI of your agent in Agent engine aka reasonning engine

#@markdown Must be in same project than agentspace

#@markdown In the form: projects/{project}/locations/{location}/reasoningEngines/{agent}

AGENT_URI = "projects/*/locations/*/reasoningEngines/*"  #@param {type:"string"}
#@markdown _How your agent will be called in Agentspace_
DISPLAY_NAME = "The Yoda search agent (updated from API)"  #@param {type:"string"}
#@markdown _The description for your agent in Agentspace_
DESCRIPTION = "This agent is using the google_search tool to help the user"  #@param {type:"string"}

os.environ["DISPLAY_NAME"] = DISPLAY_NAME
os.environ["DESCRIPTION"] = DESCRIPTION
os.environ["AGENT_URI"] = AGENT_URI


In [ ]:
%%bash

# Endpoint for Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents"
echo "API_ENDPOINT: ${API_ENDPOINT}"

# --- Creating Agent  ---
curl -s -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \
-d @- <<EOF
{
  "displayName": "${DISPLAY_NAME}",
  "description": "${DESCRIPTION}",
  "adk_agent_definition": {
    "tool_settings": {
      "tool_description": "Uses google_search"
    },
    "provisioned_reasoning_engine": {
      "reasoning_engine": "${AGENT_URI}"
    }
  }
}
EOF
echo -e "\nAgent creation attempt finished.\n"

## Remove an agent from Agentspace

In [ ]:
#@title Setting to remove an agent
#@markdown The name of your agent in Agent space
#@markdown in the form: projects/{project}/locations/{location}/collections/{collection}/engines/{engine}/assistants/{assistant}/agents/{agent}
AGENT_NAME_del = "projects/*/locations/*/collections/*/engines/*/assistants/*/agents/*"  #@param {type:"string"}
os.environ["AGENT_NAME_del"] = AGENT_NAME_del

### API documentation  
You delete a resource by sending a `DELETE` request to the agent resource identified by their name.    
Doc: https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents/delete


In [ ]:
%%bash

# Resource Endpoint in Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/${AGENT_NAME_del}"
echo "API_ENDPOINT: ${API_ENDPOINT}" # Print the API endpoint for debugging

# --- Register Agent  ---
echo "Deleting Agent: ${AGENT_NAME_del}..."
curl -s -X DELETE \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \

echo -e "\nAgent deletion attempt finished.\n"

## Deploy an agentspace agent - not working...
Not really clear what this does beyond the possibility to redeploy an existing agent.   

Use the name of the agent in agent space in the form projects/{project}/locations/{location}/collections/{collection}/engines/{engine}/assistants/{assistant}/agents/{agent}



In [ ]:
#@markdown The name of your agent in Agentspace
#@markdown in the form: projects/{project}/locations/{location}/collections/{collection}/engines/{engine}/assistants/{assistant}/agents/{agent}

AGENT_NAME_POST = "projects/*/locations/*/collections/*/engines/*/assistants/*/agents/*"  #@param {type:"string"}
os.environ["AGENT_NAME_POST"] = AGENT_NAME_POST

In [ ]:
%%bash

# Endpoint for Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/${AGENT_NAME_POST}:deploy"
echo "API_ENDPOINT: ${API_ENDPOINT}" # Print the API endpoint for debugging

# --- Deploy Agent  ---
echo "Deploy Agent: ${AGENT_NAME_POST} ..."
curl -s -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \

echo -e "\nAgent deployment attempt finished.\n"

## Get details about a published agent

### API documentation  
You get a resource details by sending a `GET` request to the agent resource identified by name.  
Doc: https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents/get

In [ ]:
#@markdown _The id of your agent in Agent Engine (number back */reasoningengines/?)_
AGENT_ID_GET = "012345678912345678"  #@param {type:"string"}
os.environ["AGENT_ID_GET"] = AGENT_ID_GET

In [ ]:
%%bash

# Endpoint for Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents/${AGENT_ID_GET}"

echo "API_ENDPOINT: ${API_ENDPOINT}" # Print the API endpoint for debugging

# --- list Agents  ---
echo "Info about agent : ${AGENT_ID_GET}..."

curl -s -X GET \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \

echo -e "\nend details.\n"

Get more (?) details with AgentView  
But I can't really see the difference between the two

In [ ]:
%%bash

# Endpoint for Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents/${AGENT_ID_GET}"

echo "API_ENDPOINT: ${API_ENDPOINT}:getAgentView" # Print the API endpoint for debugging

# --- list Agents  ---
echo "Info about agent : ${AGENT_ID_GET}..."

curl -s -X GET \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \

echo -e "\nend details.\n"

## List published agents

In [ ]:
%%bash

# Endpoint for Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents"
echo "API_ENDPOINT: ${API_ENDPOINT}" # Print the API endpoint for debugging

# --- list Agents  ---
echo "Listing Agents from: ${AS_APP}..."

curl -s -X GET \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \

echo -e "\nend of list.\n"

## Update an agent in Agentspace  
Note that you send data in the body with an "agent" incl. all required fields but you can use a mask in parameter to only update the fields you want to update.  
Example demo using an agent element and without mask

In [ ]:
#@title Setting to update an agent

#@markdown The URI of your agent in Agent engine aka reasonning engine

#@markdown Must be in same project than agentspace

#@markdown In the form: projects/{project}/locations/{location}/reasoningEngines/{agent}

AGENT_URI_PATCH = "projects/*/locations/*/reasoningEngines/*"  #@param {type:"string"}
os.environ["AGENT_URI_PATCH"] = AGENT_URI_PATCH

#@markdown _The name of your agent in Agent Engine (number back */reasoningengines/?)_
AGENT_NAME_PATCH = "0123456789012345678"  #@param {type:"string"}
#@markdown _How your agent will be called in Agentspace_
DISPLAY_NAME_patch = "The Yoda search agent (updated from API)"  #@param {type:"string"}
#@markdown _The description for your agent in Agentspace_
DESCRIPTION_patch = "This agent is using the google_search tool to help the user"  #@param {type:"string"}

os.environ["DISPLAY_NAME_patch"] = DISPLAY_NAME_patch
os.environ["DESCRIPTION_patch"] = DESCRIPTION_patch
os.environ["AGENT_NAME_PATCH"] = AGENT_NAME_PATCH

### API documentation  
You update a resource by sending a `PATCH` request to the agent resource in Agentspace .  
Doc: https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest/v1alpha/projects.locations.collections.engines.agents/patch  

In [ ]:
%%bash
# Endpoint for the agent in the Agentspace app
export API_ENDPOINT="https://discoveryengine.googleapis.com/v1alpha/projects/${GOOGLE_CLOUD_PROJECT}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents/${AGENT_NAME_PATCH}"
echo "API_ENDPOINT: ${API_ENDPOINT}" # Print the API endpoint for debugging

# --- Register Agent  ---
echo "Updating Agent: ${AGENT_NAME_PATCH}..."
curl -s -X PATCH \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${GOOGLE_CLOUD_PROJECT}" \
"${API_ENDPOINT}" \
-d @-<<EOF
{
  "displayName": "${DISPLAY_NAME_patch}",
  "description": "${DESCRIPTION_patch}",

  "adkAgentDefinition":
  {
      "toolSettings": {
        "toolDescription": "Uses google_search"
      },
      "provisionedReasoningEngine": {
        "reasoningEngine": "${AGENT_URI_PATCH}"
      }
  }
}
EOF
echo -e "\nAgent Update attempt finished.\n"